# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [ ]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [ ]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Max Temp (C)
0,0,pan de azucar,-55.2333,-34.8000,297.46,91,0,3.60,UY,1707636203,24.31
1,1,chonchi,-73.8092,-42.6128,285.44,97,100,1.01,CL,1707636203,12.29
2,2,strathmore,-113.3852,51.0501,275.82,74,68,3.09,CA,1707635678,2.67
3,3,bredasdorp,20.0403,-34.5322,294.96,50,0,7.07,ZA,1707636204,21.81
4,4,meadow lakes,-149.6011,61.6247,274.14,91,100,2.57,US,1707636204,0.99


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [ ]:
%%capture --no-display

# Configure the map plot
city_data_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    title='City Humidity',
    xlabel='Longitude',
    ylabel='Latitude',
    size = "Humidity",
    hover_cols=['City', 'Humidity'],
    frame_height=400,
    frame_width=600

)
# Display the map
city_data_map

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [84]:
desired = city_data_df.loc[(city_data_df["Max Temp (C)"] > 21) & (city_data_df["Max Temp (C)"] < 28) & (city_data_df["Cloudiness"] < 50) & (city_data_df["Wind Speed"] < 3) & (city_data_df["Humidity"] < 65) & (city_data_df["Humidity"] > 25), :]
desired.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11 entries, 38 to 571
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   City_ID       11 non-null     int64  
 1   City          11 non-null     object 
 2   Lat           11 non-null     float64
 3   Lng           11 non-null     float64
 4   Max Temp      11 non-null     float64
 5   Humidity      11 non-null     int64  
 6   Cloudiness    11 non-null     int64  
 7   Wind Speed    11 non-null     float64
 8   Country       10 non-null     object 
 9   Date          11 non-null     int64  
 10  Max Temp (C)  11 non-null     float64
dtypes: float64(5), int64(4), object(2)
memory usage: 1.0+ KB


In [85]:
# Narrow down cities that fit criteria and drop any results with null values
# Drop any rows with null values
desired = desired.dropna()

# Display sample data
desired.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Max Temp (C)
38,38,bubaque,-15.8333,11.2833,298.77,57,14,2.54,GW,1707636216,25.62
128,128,sao vicente,-46.3919,-23.9631,299.41,26,6,0.45,BR,1707636248,26.26
184,184,ormara,64.6357,25.2088,295.42,48,4,0.79,PK,1707636268,22.27
227,227,cooma,149.1333,-36.2333,296.95,35,28,2.61,AU,1707636282,23.80
401,401,kant,79.8000,27.8000,295.57,36,0,1.13,IN,1707636448,22.42


### Step 3: Create a new DataFrame called `hotel_df`.

In [86]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = desired.copy()
hotel_df = hotel_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] =""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
38,bubaque,GW,-15.8333,11.2833,57,
128,sao vicente,BR,-46.3919,-23.9631,26,
184,ormara,PK,64.6357,25.2088,48,
227,cooma,AU,149.1333,-36.2333,35,
401,kant,IN,79.8000,27.8000,36,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [61]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "limit":1,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lon},{lat},{radius}'
    params["bias"] = f'proximity:{lon},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bubaque - nearest hotel: No hotel found
sao vicente - nearest hotel: No hotel found
ormara - nearest hotel: No hotel found
cooma - nearest hotel: No hotel found
kant - nearest hotel: No hotel found
yung shue wan - nearest hotel: No hotel found
mhango - nearest hotel: No hotel found
shahabad - nearest hotel: No hotel found
san miguel - nearest hotel: No hotel found
villa gesell - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
38,bubaque,GW,-15.8333,11.2833,57,No hotel found
128,sao vicente,BR,-46.3919,-23.9631,26,No hotel found
184,ormara,PK,64.6357,25.2088,48,No hotel found
227,cooma,AU,149.1333,-36.2333,35,No hotel found
401,kant,IN,79.8000,27.8000,36,No hotel found
428,yung shue wan,HK,114.1167,22.2333,33,No hotel found
479,mhango,TZ,32.8500,-3.2833,61,No hotel found
509,shahabad,IN,79.9500,27.6500,34,No hotel found
524,san miguel,SV,-88.1833,13.4833,62,No hotel found
571,villa gesell,AR,-56.9730,-37.2639,51,No hotel found


Looking at the JSON that was returned from the API advised that the closest accommodation, let alone the closest hotel, was ~60000 meters away from the cities in my dataframe, I removed the radius = 10000 filter and looking for categories = "accommodation" as requested above so that I could complete the assignment (i.e. plot accommodations on the map in step 5.)


In [79]:
params = {
    "categories":"accommodation",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["bias"] = f'proximity:{lon},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest accommodation: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bubaque - nearest hotel: Pousada Flamingo
sao vicente - nearest hotel: Rockhopper Cottage
ormara - nearest hotel: No hotel found
cooma - nearest hotel: No hotel found
kant - nearest hotel: Svalbard
yung shue wan - nearest hotel: No hotel found
mhango - nearest hotel: Relais Talsint Hotel
shahabad - nearest hotel: Svalbard
san miguel - nearest hotel: Whichaway Camp
villa gesell - nearest hotel: Lookout Lodge


,City,Country,Lat,Lng,Humidity,Hotel Name
38,bubaque,GW,-15.8333,11.2833,57,Pousada Flamingo
128,sao vicente,BR,-46.3919,-23.9631,26,Rockhopper Cottage
184,ormara,PK,64.6357,25.2088,48,No hotel found
227,cooma,AU,149.1333,-36.2333,35,No hotel found
401,kant,IN,79.8000,27.8000,36,Svalbard
428,yung shue wan,HK,114.1167,22.2333,33,No hotel found
479,mhango,TZ,32.8500,-3.2833,61,Relais Talsint Hotel
509,shahabad,IN,79.9500,27.6500,34,Svalbard
524,san miguel,SV,-88.1833,13.4833,62,Whichaway Camp
571,villa gesell,AR,-56.9730,-37.2639,51,Lookout Lodge


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
city_data_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    frame_height=800,
    frame_width=600,
    scale = 0.05,
    
    hover_cols = ["Hotel Name", "Country"]

)

# Display the map
city_data_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)